In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#pedojoe',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(8, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

,tweetText,text
0,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi
1,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi
2,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi
3,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi
4,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi


In [4]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


,text,text2
0,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]"
1,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]"
2,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]"
3,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]"
4,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]"


In [5]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

,tweetText,text2,text3
0,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]"
1,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]"
2,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]"
3,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]"
4,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]"


In [6]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#pedojoe"
DataSet["letoltes_datum"] = today

In [7]:
DataSet.to_csv('data/pedojoe' + str(year) + str(month) + str(day) + '.csv',index=False)

In [8]:
DataSet.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,...,userTimezone,Coordinates,GeoEnabled,Language,TweetPlace,text,text2,text3,keresoszo,letoltes_datum
0,1312377746971127809,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,762,0,Twitter for iPhone,2020-10-03 13:03:32,241922327,DanStevens1103,Dan Stevens,2011-01-23 13:43:57,...,None,None,False,None,null,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]",#pedojoe,2020.10.03
1,1312324070088867840,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,762,0,Twitter Web App,2020-10-03 09:30:14,786948639197372416,ProbableCause45,"❌Wayne-Upstate,S.C.❌",2016-10-14 15:15:51,...,None,None,False,None,null,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]",#pedojoe,2020.10.03
2,1312313054554722305,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,762,0,Twitter for iPad,2020-10-03 08:46:28,2468328434,HambrickRo,Robin Hambrick,2014-04-28 22:12:44,...,None,None,False,None,null,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]",#pedojoe,2020.10.03
3,1312312478051692546,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,762,0,Twitter for iPhone,2020-10-03 08:44:11,1244745463074455552,Spankin40,Spankin,2020-03-30 21:57:10,...,None,None,False,None,null,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]",#pedojoe,2020.10.03
4,1312257095933333505,RT @JesseKellyDC: Why hasn’t Joe Biden denounced pedophilia yet? This shouldn’t be hard. Does Joe Biden support pedophilia? Is he a pedophi…,762,0,Twitter for iPad,2020-10-03 05:04:06,865636782,Maxinerunner,Maxine,2012-10-07 02:30:46,...,None,None,False,None,null,why hasnt joe biden denounced pedophilia yet this shouldnt be hard does joe biden support pedophilia is he a pedophi,"[why, hasnt, joe, biden, denounced, pedophilia, yet, this, shouldnt, be, hard, does, joe, biden, support, pedophilia, is, he, a, pedophi]","[whi, hasnt, joe, biden, denounc, pedophilia, yet, this, shouldnt, be, hard, doe, joe, biden, support, pedophilia, is, he, a, pedophi]",#pedojoe,2020.10.03
